Loading the python packages

In [3]:
import pandas as pd
from pickle import load
import plotly.graph_objects as go

Loading the Datasets from Trusted Layer

In [4]:
crypto_ai = pd.read_csv('Trusted/crypto_ai_energy.csv')
world = pd.read_csv('Trusted/world_energy.csv')

Defining the forecast range

In [7]:
years_ahead = 10
start_year = 2025
years_predict = pd.DataFrame({'Year':range(start_year,start_year+years_ahead,1)})

Loading the Models from Models folder

In [8]:
# load the model from disk
crypto_ai_model = load(open('Models/crypto_ai_forecast.sav', 'rb'))
world_model = load(open('Models/world_energy_forecast.sav', 'rb'))
#rescaledValidationX = scaler.transform(X_validation) #in case the data is scaled.
#predictions = model.predict(rescaledValidationX)
predictions_cd = crypto_ai_model.get_forecast(steps = years_ahead, exog = years_predict)
predictions_w = world_model.get_forecast(steps = years_ahead, exog = years_predict)

In [5]:
predictions.conf_int()

,lower Total Consumption,upper Total Consumption
11,577.534999,655.699092
12,616.170363,702.752913
13,639.699136,729.585392
14,657.066002,746.262077
15,667.070498,759.225049
16,674.859831,776.337244
17,687.361545,800.665074
18,710.907732,833.605746
19,748.246540,875.423315
20,798.015876,925.699462


In [6]:
predictions.predicted_mean

11    616.617046
12    659.461638
13    684.642264
14    701.664039
15    713.147773
16    725.598538
17    744.013309
18    772.256739
19    811.834928
20    861.857669
Name: predicted_mean, dtype: float64

In [ ]:
# Create Plotly figure
fig = go.Figure()

# Observed data
fig.add_trace(go.Scatter(
    x=df.index, y=df['Value'],
    mode='lines', name='Observed',
    line=dict(color='black')
))

# Mean forecast
fig.add_trace(go.Scatter(
    x=mean_forecast.index, y=mean_forecast,
    mode='lines', name='Forecast',
    line=dict(color='blue')
))

# Confidence interval (as shaded area)
fig.add_trace(go.Scatter(
    x=list(mean_forecast.index) + list(mean_forecast.index[::-1]),
    y=list(conf_int.iloc[:, 1]) + list(conf_int.iloc[:, 0][::-1]),
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name='95% CI'
))

fig.update_layout(
    title="ARIMA Forecast with Confidence Interval",
    xaxis_title="Date",
    yaxis_title="Value",
    template="plotly_white"
)

fig.show()